In [1]:
import pandas as pd

In [3]:
ori = pd.read_json('replace_origin (1).json')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
ori

,text,label
0,"ORG uy tín số 1, check mã Viên uống trắng da r...",2
1,"Ở LOC thiếu gì chỗ bán, dại dột mua online của...",0
2,"PER tư vấn sai bét, làm mình uống sai liều lượng.",0
3,Da dẻ hồng hào hẳn lên nhờ uống Viên sủi của O...,2
4,Mua Nước hồng sâm tại ORG thấy cũng bình thườn...,1
...,...,...
2995,"Mới nhận được Nước hồng sâm từ ORG sáng nay, c...",1
2996,Uống hết 1 hộp Men vi sinh dạng ống rồi mà thấ...,1
2997,"ORG giao thiếu Viên khớp Glucosamine, nhắn tin...",0
2998,"Hàng về tới LOC rồi, đóng gói kỹ, còn chất lượ...",1


In [6]:
from sklearn.model_selection import train_test_split

# train (70%), validation (15%), test (15%)
train_df, temp_df = train_test_split(ori, test_size=0.3, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2100 entries, 611 to 860
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2100 non-null   object
 1   label   2100 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 49.2+ KB


In [8]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 92.2 MB/s eta 0:00:00


In [ ]:
!pip install gensim transformers torch

In [9]:
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from gensim.utils import simple_preprocess
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

from transformers import get_linear_schedule_with_warmup, AutoTokenizer, AutoModel, logging


import warnings
warnings.filterwarnings("ignore")
logging.set_verbosity_error()

In [10]:
def seed_set(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed_set(18)

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
EPOCHS = 5
n = 5

In [12]:
train_df = pd.concat([train_df, valid_df], ignore_index=True)
skf = StratifiedKFold(n_splits=n)
for fold, (_, val_) in enumerate(skf.split(X=train_df, y=train_df.label)):
    train_df.loc[val_, "kfold"] = fold

In [13]:
train_df.sample(5)

,text,label,kfold
2518,Mua Viên khớp Glucosamine tại ORG thấy cũng bì...,1,4.0
1826,"Ở LOC thiếu gì chỗ bán, dại dột mua online của...",0,3.0
1272,Đặt hỏa tốc tới LOC mà 3 ngày mới nhận được hàng.,0,2.0
1857,"Mua Thuốc giảm đau theo review trên mạng, để t...",1,3.0
1268,Mới nhận được Men vi sinh dạng ống từ ORG sáng...,1,2.0


In [15]:
Model = AutoModel.from_pretrained("vinai/phobert-base")

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [16]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")


vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

# Xử lý dữ liệu



In [20]:
class SentimentDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=120):
        self.df = df
        self.max_len = max_len
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        text, label = self.get_input_data(row)
        encoding = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding=True,  # Set padding to True
            return_attention_mask=True,
            return_token_type_ids=False,
        )

        # Manually truncate or pad the input_ids and attention_mask
        input_ids = encoding['input_ids'] + [0] * (self.max_len - len(encoding['input_ids']))
        attention_mask = encoding['attention_mask'] + [0] * (self.max_len - len(encoding['attention_mask']))


        return {
            'text': text,
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_masks': torch.tensor(attention_mask, dtype=torch.long),
            'targets': torch.tensor(label, dtype=torch.long),
        }


    def labelencoder(self,text):
        if text=='NEG':
            return 0
        elif text=='NEU':
            return 1
        elif text=='POS':
            return 2

    def get_input_data(self, row):
        #{remove icon, special character, lower}
        text = row['text']
        text = ' '.join(simple_preprocess(text))
        label = self.labelencoder(row['label'])

        return text, label

In [21]:
ex ={'text': "Chúc bạn một ngày tốt lành", 'label': 'NEU'}
df = pd.DataFrame([ex])
ex_sen = SentimentDataset(df, tokenizer, max_len=10)
sample = ex_sen.__getitem__(0)
sample

[ ]
ex ={'text': "Chúc bạn một ngày tốt lành", 'label': 'NEU'}
df = pd.DataFrame([ex])
ex_sen = SentimentDataset(df, tokenizer, max_len=10)
sample = ex_sen.__getitem__(0)
sample



{'text': 'chúc bạn một ngày tốt lành',
 'input_ids': tensor([   0, 3788,   88,   16,   43,  167, 4446,    2,    0,    0]),
 'attention_masks': tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0]),
 'targets': tensor(1)}

In [22]:
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = Model

        self.drop = nn.Dropout(p=0.3)

        self.fc = nn.Linear(self.bert.config.hidden_size, n_classes)
        nn.init.normal_(self.fc.weight, std=0.02)
        nn.init.normal_(self.fc.bias, 0)

    def forward(self, input_ids, attention_mask):
        last_hidden_state, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False
        )

        x = self.drop(output)
        x = self.fc(x)
        return x

In [23]:
def train(model, criterion, optimizer, train_loader, lr_scheduler=None, device='cuda'):
    model.train()
    losses = []
    correct = 0

    for data in train_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        targets = data['targets'].to(device)

        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss = criterion(outputs, targets)
        _, pred = torch.max(outputs, dim=1)

        correct += torch.sum(pred == targets)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

    accuracy = correct.double() / len(train_loader.dataset)
    mean_loss = np.mean(losses)
    print(f'Train Accuracy: {accuracy} Loss: {mean_loss}')

In [24]:
def eval(test_data=False):
    model.eval()
    losses = []
    correct = 0

    with torch.no_grad():
        data_loader = test_loader if test_data else valid_loader
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            targets = data['targets'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, pred = torch.max(outputs, dim=1)

            loss = criterion(outputs, targets)
            correct += torch.sum(pred == targets)
            losses.append(loss.item())

    if test_data:
        accuracy = correct.double() / len(test_loader.dataset)
        print(f'Test Accuracy: {accuracy} Loss: {np.mean(losses)}')
        return accuracy
    else:
        accuracy = correct.double() / len(valid_loader.dataset)
        print(f'Valid Accuracy: {accuracy} Loss: {np.mean(losses)}')
        return accuracy

In [25]:
def prepare_loaders(df, fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    train_dataset = SentimentDataset(df_train, tokenizer, max_len=120)
    valid_dataset = SentimentDataset(df_valid, tokenizer, max_len=120)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
    valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=True, num_workers=2)

    return train_loader, valid_loader

In [27]:

import torch
from torch.utils.data import Dataset


train_df = train_df.dropna(subset=['label'])
train_df['label'] = train_df['label'].astype(int)


class SentimentDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=120):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        text = str(row['text'])
        # Lấy nhãn an toàn
        try:
            label = int(row['label'])
        except:
            label = 1

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(label, dtype=torch.long)
        }
print(" Đã cập nhật xong Class và Dữ liệu!")

 Đã cập nhật xong Class và Dữ liệu!


In [28]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Create a folder in Google Drive for saving models
drive_folder = '/content/drive/MyDrive/small2_data/'
!mkdir "{drive_folder}"

for fold in range(skf.n_splits):
    print(f'Fold: {fold+1}')
    train_loader, valid_loader = prepare_loaders(train_df, fold=fold)
    model = SentimentClassifier(n_classes=3).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=2e-5)

    lr_scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=0,
                num_training_steps=len(train_loader)*EPOCHS
            )
    best_acc = 0
    for epoch in range(EPOCHS):
        print(f'Epoch {epoch+1}/{EPOCHS}')
        print('-'*30)

        train(model, criterion, optimizer, train_loader)
        val_acc = eval()

        if val_acc > best_acc:
            # Save model to the created folder in Google Drive
            torch.save(model.state_dict(), f'{drive_folder}phobert_fold{fold+1}.pth')
            best_acc = val_acc

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive/MyDrive/small2_data/’: File exists
Fold: 1
Epoch 1/5
------------------------------
Train Accuracy: 0.8803921568627451 Loss: 0.3885539670372964
Valid Accuracy: 1.0 Loss: 0.007516637080698274
Epoch 2/5
------------------------------
Train Accuracy: 0.9995098039215686 Loss: 0.007359664808973321
Valid Accuracy: 1.0 Loss: 0.002364877007494215
Epoch 3/5
------------------------------
Train Accuracy: 1.0 Loss: 0.0025810693741732393
Valid Accuracy: 1.0 Loss: 0.0012251769258000422
Epoch 4/5
------------------------------
Train Accuracy: 1.0 Loss: 0.0014881100323691498
Valid Accuracy: 1.0 Loss: 0.0007760265780234477
Epoch 5/5
------------------------------
Train Accuracy: 1.0 Loss: 0.0009959161734514055
Valid Accuracy: 1.0 Loss: 0.0005419978460849961
Fold: 2
Epoch 1/5
------------------------------
Train Accuracy: 0.97009

In [29]:
def test(data_loader):
    models = []
    for fold in range(skf.n_splits):
        model = SentimentClassifier(n_classes=3)
        model.to(device)

        # Load the model from Google Drive
        drive_folder = '/content/drive/MyDrive/small2_data/'
        model_path = f'{drive_folder}phobert_fold{fold+1}.pth'
        model.load_state_dict(torch.load(model_path))
        model.eval()
        models.append(model)

    texts = []
    predicts = []
    predict_probs = []
    real_values = []

    for data in data_loader:
        text = data['text']
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        targets = data['targets'].to(device)

        total_outs = []
        for model in models:
            with torch.no_grad():
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )
                total_outs.append(outputs)

        total_outs = torch.stack(total_outs)
        _, pred = torch.max(total_outs.mean(0), dim=1)
        texts.extend(text)
        predicts.extend(pred)
        predict_probs.extend(total_outs.mean(0))
        real_values.extend(targets)

    predicts = torch.stack(predicts).cpu()
    predict_probs = torch.stack(predict_probs).cpu()
    real_values = torch.stack(real_values).cpu()
    print(classification_report(real_values, predicts))
    return real_values, predicts

In [30]:
test_dataset = SentimentDataset(test_df, tokenizer, max_len=50)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=2)
real_values, predicts = test(test_loader)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       159
           1       1.00      1.00      1.00       156
           2       1.00      1.00      1.00       135

    accuracy                           1.00       450
   macro avg       1.00      1.00      1.00       450
weighted avg       1.00      1.00      1.00       450



In [31]:
# Định nghĩa danh sách tên nhãn theo thứ tự 0, 1, 2
class_names = ['Tiêu cực', 'Trung tính', 'Tích cực']

In [36]:
def infer(text, tokenizer, max_len=120):

    models = []
    for fold in range(skf.n_splits):
        model = SentimentClassifier(n_classes=3)
        model.to(device)

        # Load the model from Google Drive
        drive_folder = '/content/drive/MyDrive/small2_data/'
        model_path = f'{drive_folder}phobert_fold{fold+1}.pth'
        model.load_state_dict(torch.load(model_path))
        model.eval()
        models.append(model)


    encoded_review = tokenizer.encode_plus(
        text,
        max_length=max_len,
        truncation=True,
        add_special_tokens=True,
        padding='max_length',
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='pt',
    )

    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)

    output = model(input_ids, attention_mask)
    _, y_pred = torch.max(output, dim=1)

    print(f'Text: {text}')
    print(f'Sentiment: {class_names[y_pred]}')

In [37]:
infer('Đồ mua từ ORC này cũng được không khá tệ tuy nhiên có vài vấn đề cần khắc phục', tokenizer)

Text: Đồ mua từ ORC này cũng được không khá tệ tuy nhiên có vài vấn đề cần khắc phục
Sentiment: Trung tính
